<a href="https://colab.research.google.com/github/dat810/tt2_things/blob/main/present_panic/bomb_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prestige Event Bomb Game

Exploring the following:
- Average scissors needed per board
- Average boards completed for the event

So that you are prepared for the RNG 🤣

## Setting Up
Load libraries

In [ ]:
import random
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import pandas as pd
import multiprocessing as mp
import os
import requests

Load the dataset (as of 7.5.2)

In [ ]:
!git clone https://github.com/rawrzcookie/TT2_CSV.git

Dataset already exists at HolidayEventBombGameLevelInfo.csv


In [ ]:
!cp TT2_CSV/csv/HolidayEventBombGameLevelInfo.csv .

In [ ]:
level_info = pd.read_csv('HolidayEventBombGameLevelInfo.csv')

Define a Board

In [ ]:
class Board:
    def __init__(self,dim,bombs,target,diff_idx):
        self.dim = dim
        self.bombs = bombs
        self.target = target
        self.boxes = self.dim[0]*self.dim[1]

        idx_to_diff = {
            0: 'Easy', 1: 'Hard', 2: 'Impossible'
        }
        self.diff = idx_to_diff.get(diff_idx)


    def __str__(self):
        return (
            f"Board size: {self.dim[0]}x{self.dim[1]}\n"
            f"Bombs count: {self.bombs}\n"
            f"Streak needed: {self.target}\n"
            f"Difficulty: {self.diff}"
        )

Define a simulator for a board. Return the number of scissors used.



In [ ]:
%%writefile bomb_game_sim.c
#include <stdlib.h>
#include <time.h>

int bomb_game_sim (int boxes, int bombs, int target) {
    int g = boxes - bombs;
    int total_scissors = 0;
    int streak = 0;
    int *board = (int *)malloc(boxes*sizeof(int));
    int i;

    // Initialize the board: 0 for bombs, 1 for corrects
    for (i=0;i<bombs;i++) {
        board[i] = 0; // bombs
    }
    for (i=bombs;i<boxes;i++) {
        board[i] = 1; // corrects
    }

    srand((unsigned int)time(NULL)+rand());

    while (streak < target) {
        int index = rand() % boxes;
        total_scissors++;

        if (board[index] == 1) { // corrects
            streak++;
            board[index] = board[--boxes]; // Remove corrects and shrink board
        } else { // troll
            streak = 0;
            bombs--;
            boxes = bombs + g; // Reset board
            for (i = 0; i < bombs; i++) board[i] = 0; // bombs
            for (i = bombs; i < boxes; i++) board[i] = 1; // corrects
        }
    }

    free(board);
    return total_scissors;
}

Define a simulator for an event run. Return the number of boards completed.

Parse the board information into a dataframe, and create the `Board` objects.

In [ ]:
parsed = level_info[['Rows', 'Columns', 'BombsOnBoard', 'SuccessesNeededToWin', 'ChestIconIndex']]

boards = []
for row in parsed.itertuples(index=False):
    boards.append(Board([row[0],row[1]],row[2],row[3],row[4]))

## Average Scissors Needed per Board

## Average Boards Completed in an Event

## Notebook Information
- Notebook content created by dat
- Dataset from rawrzcookie, who also fetched the CSV files here: https://github.com/rawrzcookie/TT2_CSV